# Analyze the experimental results and generate a report

#### Authors

* Juan Carlos Alfaro Jiménez
* Juan Ángel Aledo Sánchez
* José Antonio Gámez Martín

In this notebook, we analyze the experimental results and generate a report (`HTML` format). Below, we detail the steps.

## 1. Arguments

First, we add the command line arguments:

In [ ]:
library(argparser)

In [ ]:
description <- "Analysis of experimental results and report generation."

In [ ]:
parser <- arg_parser(description)

* The path to the tables:

In [ ]:
arg <- "--source"

In [ ]:
default <- "tables"

In [ ]:
help <- "Path to the tables"

In [ ]:
parser <- add_argument(parser, arg, help, default = default)

* The path to the rendered file:

In [ ]:
arg <- "--destination"

In [ ]:
default <- "reports"

In [ ]:
help <- "Path to the rendered file"

In [ ]:
parser <- add_argument(parser, arg, help, default = default)

* The name of the target output variable:

In [ ]:
arg <- "--output"

In [ ]:
default <- "test_score"

In [ ]:
help <- "Name of the target output variable"

In [ ]:
parser <- add_argument(parser, arg, help, default = default)

* The optimization strategy of the target output variable:

In [ ]:
arg <- "--rank"

In [ ]:
default <- "max"

In [ ]:
help <- "Optimization strategy of the target output variable"

In [ ]:
parser <- add_argument(parser, arg, help, default = default)

* The number of decimal digits for the numeric output

In [ ]:
arg <- "--digits"

In [ ]:
default <- 3

In [ ]:
help <- "Number of decimal digits for the numeric output"

In [ ]:
parser <- add_argument(parser, arg, help, default = default)

* The short title for the document:

In [ ]:
arg <- "--title"

In [ ]:
help <- "Short title for the document"

In [ ]:
default <- "Report"

In [ ]:
parser <- add_argument(parser, arg, help, default = default)

* The significance level used for the testing procedure

In [ ]:
arg <- "--alpha"

In [ ]:
default <- 0.05

In [ ]:
help <- "Significance level used for the testing procedure"

In [ ]:
parser <- add_argument(parser, arg, help, default = default)

* The subset of methods to filter:

In [ ]:
arg <- "--methods"

In [ ]:
default <- ".*"

In [ ]:
help <- "Subset of methods to filter"

In [ ]:
parser <- add_argument(parser, arg, help, default = default)

* The subset of problems to filter:

In [ ]:
arg <- "--problems"

In [ ]:
default <- ".*"

In [ ]:
help <- "Subset of problems to filter"

In [ ]:
parser <- add_argument(parser, arg, help, default = default)

Now, we parse the command line arguments:

In [ ]:
argv <- readLines(con = "arguments.txt", n = 1)

In [ ]:
argv <- strsplit(argv, split = " ")

In [ ]:
argv <- parse_args(parser, argv = argv)

And rename the variables:

In [ ]:
source <- argv$source

In [ ]:
source

In [ ]:
destination <- argv$destination

In [ ]:
destination

In [ ]:
output <- argv$output

In [ ]:
output

In [ ]:
rank <- argv$rank

In [ ]:
rank

In [ ]:
title <- argv$title

In [ ]:
title

In [ ]:
digits <- argv$digits

In [ ]:
digits

In [ ]:
alpha <- argv$alpha

In [ ]:
alpha

In [ ]:
methods <- argv$methods

In [ ]:
methods

In [ ]:
problems <- argv$problems

In [ ]:
problems

## 2. Load

Second, we get the file with the table (`source`):

In [ ]:
source <- file.path("work", source, output, "mean.csv")

In [ ]:
data <- read.csv(source, row.names = 1, check.names = FALSE)

And include a column with the `methods`:

In [ ]:
data <- cbind(rownames(data), data)

In [ ]:
colnames(data)[1] <- "method"

Then, we initialize the report document (`title`) and create the experiment object from a tabular representation:

In [ ]:
library(exreport)

In [ ]:
report <- exreport(title)

In [ ]:
experiment <- expCreateFromTable(data, output, title)

Finally, we filter the methods (`methods`) and datasets (`datasets`):

In [ ]:
library(stringr)

In [ ]:
rows <- rownames(data)

In [ ]:
cols <- colnames(data)

In [ ]:
methods <- str_subset(rows, methods)

In [ ]:
problems <- str_subset(cols, problems)

In [ ]:
subset <- list(method = methods, problem = problems)

In [ ]:
experiment <- expSubset(experiment, subset)

In [ ]:
experiment <- expInstantiate(experiment)

## 3. Analyze

Third, we summarize the experiment with a table and a plot for the given target output variable (`output`) according with the optimization strategy (`rank`) and number of decimal digits (`digits`):

In [ ]:
tabular_exp_summary <- tabularExpSummary(experiment, output, rank, digits = digits)

In [ ]:
plot_exp_summary <- plotExpSummary(experiment, output, columns = 5, freeScale = TRUE)

In [ ]:
report <- exreportAdd(report, tabular_exp_summary)

In [ ]:
report <- exreportAdd(report, plot_exp_summary)

Now, we perform a multiple comparison statistical test for the given experiment. In particular, we apply a *Friedman test* and a *post-hoc test* with the *Shaffer procedure*:

In [ ]:
test_multiple_pairwise <- testMultiplePairwise(experiment, output, rank, alpha)

In [ ]:
tabular_test_pairwise <- tabularTestPairwise(test_multiple_pairwise)

In [ ]:
report <- exreportAdd(report, test_multiple_pairwise)

In [ ]:
report <- exreportAdd(report, tabular_test_pairwise)

And the *Holm procedure*:

In [ ]:
metrics <- c("rank", "pvalue", "wtl")

In [ ]:
test_multiple_control <- testMultipleControl(experiment, output, rank, alpha)

In [ ]:
tabular_test_summary <- tabularTestSummary(test_multiple_control, metrics)

In [ ]:
plot_rank_distribution <- plotRankDistribution(test_multiple_control)

In [ ]:
report <- exreportAdd(report, test_multiple_control)

In [ ]:
report <- exreportAdd(report, tabular_test_summary)

In [ ]:
report <- exreportAdd(report, plot_rank_distribution)

## 4. Generate

Fourth, we generate the report (`destination`):

In [ ]:
destination <- file.path("work", destination, output)

In [ ]:
dir.create(destination, showWarnings = FALSE, recursive = TRUE)

In [ ]:
exreportRender(report, destination, target = "html", safeMode = FALSE, visualize = FALSE)

Finally, we write the destination directory in a file for the `HTML` export:

In [ ]:
writeLines(destination, "destination.txt")